# Iterators and Generators

## Iterators

An `iterator` is nothing more than a container object what implements the iterator protocol. It's based on two method:

* `__next__`, which returns the next item of container
* `__iter__`, which returns the iterator itself

Iterators can be created with a sequence using the `iter` built-in function.

In [ ]:
i = iter('abc')
next(i)
next(i)
next(i)
next(i)

In [ ]:
class MyIterator(object):
    def __init__(self, step):   
        self.step = step
    def __next__(self):
        """Returns the next element."""
        if self.step == 0:
            raise StopIteration
        self.step -= 1
        return self.step
    def __iter__(self):
        """Returns the iterator itself."""
        return self
        
for el in MyIterator(4):
    print(el)

## Generators

In [ ]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield b
        a, b = b, a + b

fib = fibonacci()

[next(fib) for i in range(10)]

`generator` can interact with the code called with the `next` method. `yield` becomes an expression, and a value can be passed along with a new method called `send`.

In [ ]:
def psychologist():
    print('Please tell me your problems')
    while True:
        answer = (yield)
        if answer is not None:
            if answer.endswith('?'):
                print ("Don't ask yourself too much questions")
            elif 'good' in answer:
                print("A that's good, go on")
            elif 'bad' in answer:
                print("Don't be so negative")

free = psychologist()
next(free)
free.send('I feel bad')
free.send("Why I shouldn't ?")
free.send("ok then i should find what is good for me")